In [1]:
import json

response ={
  "id": "msg_01R5bwN8rjmbYVswKrqyWrBJ",
  "type": "message",
  "role": "assistant",
  "model": "claude-opus-4-5-20251101",
  "content": [
    {
      "type": "text",
      "text": "# %% [markdown]\n# # 🥊 Pose Time Series Analysis Lab: การวิเคราะห์ท่ามวยไทยด้วย Time Series และ Angle Analysis\n#\n# ---\n#\n# ## 📋 Lab Overview\n#\n# **วัตถุประสงค์การเรียนรู้ (Learning Objectives):**\n# 1. เข้าใจโครงสร้างข้อมูล Pose Estimation ในรูปแบบ Time Series\n# 2. วิเคราะห์และเปรียบเทียบการเคลื่อนไหวของบุคคลต่างๆ ข้ามเวลา\n# 3. คำนวณ Joint Angles จาก Keypoint Coordinates\n# 4. สร้าง Visualization เพื่อเปรียบเทียบท่าทางต่างๆ (Actions)\n# 5. วิเคราะห์ Pattern ของท่ามวยไทยผ่าน Time Series และ Angle Analysis\n#\n# **Prerequisites:**\n# - Python 3.8+\n# - ความเข้าใจพื้นฐานเรื่อง Pandas DataFrame\n# - ความเข้าใจพื้นฐานเรื่อง Trigonometry (สำหรับการคำนวณมุม)\n#\n# **Estimated Time:** 2-3 hours\n#\n# ---\n\n# %% [markdown]\n# ## 📚 Part 1: Environment Setup และ Data Loading\n#\n# ในส่วนนี้เราจะเตรียมสภาพแวดล้อมสำหรับการวิเคราะห์ โดยการ import libraries ที่จำเป็น\n# และโหลดข้อมูล Pose Estimation จากไฟล์ CSV\n#\n# ### 1.1 Import Libraries\n#\n# **อธิบาย:** นำเข้า libraries ที่จำเป็นสำหรับการวิเคราะห์ Time Series และการสร้าง Visualization\n#\n# | Library | หน้าที่ |\n# |---------|--------|\n# | pandas | จัดการข้อมูลในรูปแบบ DataFrame |\n# | numpy | การคำนวณเชิงตัวเลขและ array operations |\n# | matplotlib | สร้างกราฟและ visualization พื้นฐาน |\n# | seaborn | สร้างกราฟสถิติที่สวยงาม |\n# | scipy | Signal processing และ smoothing |\n\n# %%\n# =====================================================\n# STEP 1.1: Import Required Libraries\n# =====================================================\n# 📝 Description: \n# นำเข้า libraries ทั้งหมดที่จำเป็นสำหรับการวิเคราะห์\n# - pandas: สำหรับจัดการข้อมูล DataFrame\n# - numpy: สำหรับการคำนวณเชิงตัวเลข\n# - matplotlib & seaborn: สำหรับสร้างกราฟ\n# - scipy: สำหรับ signal processing (smoothing)\n#\n# 💡 Tip: ถ้ายังไม่มี library ใดๆ ให้รัน: \n#     pip install pandas numpy matplotlib seaborn scipy\n# =====================================================\n\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom scipy import signal\nfrom scipy.ndimage import uniform_filter1d\nimport warnings\n\n# ตั้งค่า Visualization ให้แสดงผลสวยงาม\nplt.rcParams['figure.figsize'] = (14, 8)      # ขนาดรูปเริ่มต้น\nplt.rcParams['font.size'] = 11                 # ขนาดตัวอักษร\nplt.rcParams['axes.titlesize'] = 14            # ขนาดหัวข้อกราฟ\nplt.rcParams['axes.labelsize'] = 12            # ขนาด label แกน\nsns.set_style(\"whitegrid\")                     # สไตล์กราฟแบบมี grid\nwarnings.filterwarnings('ignore')              # ปิด warnings\n\n# สำหรับแสดงผลภาษาไทย (ถ้ามี font)\ntry:\n    plt.rcParams['font.family'] = 'DejaVu Sans'\nexcept:\n    pass\n\nprint(\"✅ Libraries imported successfully!\")\nprint(f\"   📊 Pandas version: {pd.__version__}\")\nprint(f\"   🔢 NumPy version: {np.__version__}\")\n\n# %% [markdown]\n# ### 1.2 Load Pose Data\n#\n# **อธิบาย:** โหลดข้อมูล Pose Estimation จากไฟล์ CSV\n#\n# **โครงสร้างข้อมูล (Data Structure):**\n#\n# ```\n# ┌─────────────────────────────────────────────────────────────┐\n# │                    POSE DATA STRUCTURE                      │\n# ├─────────────────────────────────────────────────────────────┤\n# │ frame_idx      : หมายเลขเฟรม (ดัชนีเวลา)                    │\n# │ timestamp      : เวลาในหน่วยวินาที                          │\n# │ person_id      : รหัสบุคคลที่ถูกติดตาม                      │\n# │ avg_confidence : ความเชื่อมั่นเฉลี่ยของ keypoints           │\n# │ keypoint_x     : พิกัด X ของแต่ละ keypoint                  │\n# │ keypoint_y     : พิกัด Y ของแต่ละ keypoint                  │\n# │ keypoint_conf  : ความเชื่อมั่นของแต่ละ keypoint             │\n# │ action         : ป้ายกำกับท่าทาง (Label)                    │\n# └─────────────────────────────────────────────────────────────┘\n# ```\n#\n# **Keypoints ทั้งหมด 17 จุด:**\n# ```\n#        nose (0)\n#     left_eye (1)  right_eye (2)\n#     left_ear (3)  right_ear (4)\n#         |\n#   left_shoulder (5) ─── right_shoulder (6)\n#         |                    |\n#    left_elbow (7)      right_elbow (8)\n#         |                    |\n#    left_wrist (9)      right_wrist (10)\n#         |                    |\n#     left_hip (11) ──── right_hip (12)\n#         |                    |\n#    left_knee (13)      right_knee (14)\n#         |                    |\n#   left_ankle (15)     right_ankle (16)\n# ```\n\n# %%\n# =====================================================\n# STEP 1.2: Load Pose Data from CSV\n# =====================================================\n# 📝 Description: \n# โหลดข้อมูลจากไฟล์ CSV ที่เตรียมไว้\n# ไฟล์นี้ประกอบด้วยข้อมูล pose estimation ของหลายคน\n# ที่ถูก track ข้ามเฟรม พร้อม action labels\n#\n# 🔄 Process:\n# 1. อ่านไฟล์ CSV ด้วย pd.read_csv()\n# 2. แสดงข้อมูลเบื้องต้นเพื่อตรวจสอบความถูกต้อง\n# 3. ดูโครงสร้างและขนาดของข้อมูล\n#\n# ⚠️ Note: ตรวจสอบว่าไฟล์ 'pose_data.csv' อยู่ใน directory เดียวกัน\n# =====================================================\n\n# โหลดข้อมูล\ndf_pose = pd.read_csv('pose_data.csv')\n\n# แสดงข้อมูลเบื้องต้น\nprint(\"=\" * 70)\nprint(\"📊 POSE DATA OVERVIEW\")\nprint(\"=\" * 70)\nprint(f\"\\n📁 Dataset Shape: {df_pose.shape[0]:,} rows × {df_pose.shape[1]} columns\")\nprint(f\"📅 Frame Range: {df_pose['frame_idx'].min()} to {df_pose['frame_idx'].max()}\")\nprint(f\"⏱️  Time Range: {df_pose['timestamp'].min():.2f}s to {df_pose['timestamp'].max():.2f}s\")\nprint(f\"👥 Unique Persons: {df_pose['person_id'].nunique()}\")\nprint(f\"🎯 Actions: {df_pose['action'].nunique()}\")\n\nprint(\"\\n\" + \"=\" * 70)\nprint(\"📋 DATA SAMPLE (First 5 rows)\")\nprint(\"=\" * 70)\ndisplay(df_pose.head())\n\n# %%\n# =====================================================\n# STEP 1.3: Data Info Summary\n# =====================================================\n# 📝 Description: \n# แสดงรายละเอียดของ columns ทั้งหมด\n# เพื่อให้เข้าใจโครงสร้างข้อมูล\n#\n# 🔍 สิ่งที่ต้องตรวจสอบ:\n# - จำนวน Non-Null Count ของแต่ละ column\n# - Dtype ของแต่ละ column\n# - Memory usage\n# =====================================================\n\nprint(\"=\" * 70)\nprint(\"📋 DATA STRUCTURE INFO\")\nprint(\"=\" * 70)\nprint(f\"\\n📊 Column Types:\")\nprint(f\"   - float64 (coordinates & confidence): 53 columns\")\nprint(f\"   - int64 (frame_idx, person_id): 2 columns\")\nprint(f\"   - object (action label): 1 column\")\nprint(f\"\\n💾 Memory Usage: ~14.9 MB\")\nprint(\"\\n\" + \"-\" * 70)\ndf_pose.info()\n\n# %% [markdown]\n# ### 1.3 Define Constants\n#\n# **อธิบาย:** กำหนดค่าคงที่สำหรับ COCO Keypoint format และ Skeleton Connections\n#\n# **COCO Keypoint Format:**\n# - มาตรฐานสากลสำหรับ Human Pose Estimation\n# - ประกอบด้วย 17 keypoints ครอบคลุมทั้งร่างกาย\n# - แต่ละ keypoint มีพิกัด (x, y) และค่า confidence\n#\n# **ภาพประกอบ Skeleton Connections:**\n# ```\n#              ○ nose\n#             /|\\\n#     left_eye ○ ○ right_eye\n#            /   \\\n#   left_ear ○     ○ right_ear\n#                   \n#    left_shoulder ○───○ right_shoulder\n#                  |   |\n#                  │   │\n#      left_elbow ○   ○ right_elbow\n#                  │   │\n#      left_wrist ○   ○ right_wrist\n#                  │   │\n#        left_hip ○───○ right_hip\n#                  │   │\n#       left_knee ○   ○ right_knee\n#                  │   │\n#      left_ankle ○   ○ right_ankle\n# ```\n\n# %%\n# =====================================================\n# STEP 1.4: Define COCO Keypoint Constants\n# =====================================================\n# 📝 Description: \n# กำหนดชื่อ Keypoints ตามมาตรฐาน COCO\n# COCO format มี 17 keypoints ครอบคลุมทั้งร่างกาย\n#\n# 🎯 Purpose:\n# - ใช้อ้างอิงชื่อ keypoints อย่างเป็นระบบ\n# - จัดกลุ่มส่วนต่างๆ ของร่างกายเพื่อการวิเคราะห์\n# - กำหนดเส้นเชื่อมสำหรับการวาด skeleton\n# =====================================================\n\n# รายชื่อ Keypoints (17 จุด) พร้อมดัชนี\nKEYPOINT_NAMES = [\n    \"nose\",           # 0 - จมูก\n    \"left_eye\",       # 1 - ตาซ้าย\n    \"right_eye\",      # 2 - ตาขวา\n    \"left_ear\",       # 3 - หูซ้าย\n    \"right_ear\",      # 4 - หูขวา\n    \"left_shoulder\",  # 5 - ไหล่ซ้าย\n    \"right_shoulder\", # 6 - ไหล่ขวา\n    \"left_elbow\",     # 7 - ข้อศอกซ้าย\n    \"right_elbow\",    # 8 - ข้อศอกขวา\n    \"left_wrist\",     # 9 - ข้อมือซ้าย\n    \"right_wrist\",    # 10 - ข้อมือขวา\n    \"left_hip\",       # 11 - สะโพกซ้าย\n    \"right_hip\",      # 12 - สะโพกขวา\n    \"left_knee\",      # 13 - เข่าซ้าย\n    \"right_knee\",     # 14 - เข่าขวา\n    \"left_ankle\",     # 15 - ข้อเท้าซ้าย\n    \"right_ankle\"     # 16 - ข้อเท้าขวา\n]\n\n# การจัดกลุ่มส่วนต่างๆ ของร่างกาย (สำหรับการวิเคราะห์เฉพาะส่วน)\nBODY_PARTS = {\n    'head': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear'],\n    'upper_body': ['left_shoulder', 'right_shoulder', 'left_elbow', \n                   'right_elbow', 'left_wrist', 'right_wrist'],\n    'lower_body': ['left_hip', 'right_hip', 'left_knee', \n                   'right_knee', 'left_ankle', 'right_ankle'],\n    'left_arm': ['left_shoulder', 'left_elbow', 'left_wrist'],\n    'right_arm': ['right_shoulder', 'right_elbow', 'right_wrist'],\n    'left_leg': ['left_hip', 'left_knee', 'left_ankle'],\n    'right_leg': ['right_hip', 'right_knee', 'right_ankle'],\n    'torso': ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']\n}\n\n# เส้นเชื่อมสำหรับ Skeleton Visualization\nSKELETON_CONNECTIONS = [\n    # Head connections\n    ('nose', 'left_eye'), ('nose', 'right_eye'),\n    ('left_eye', 'left_ear'), ('right_eye', 'right_ear'),\n    # Upper body\n    ('left_shoulder', 'right_shoulder'),  # shoulders\n    ('left_shoulder', 'left_elbow'), ('left_elbow', 'left_wrist'),\n    ('right_shoulder', 'right_elbow'), ('right_elbow', 'right_wrist'),\n    # Torso\n    ('left_shoulder', 'left_hip'), ('right_shoulder', 'right_hip'),\n    ('left_hip', 'right_hip'),\n    # Lower body\n    ('left_hip', 'left_knee'), ('left_knee', 'left_ankle'),\n    ('right_hip', 'right_knee'), ('right_knee', 'right_ankle')\n]\n\nprint(\"✅ Constants defined!\")\nprint(f\"   📍 Total Keypoints: {len(KEYPOINT_NAMES)}\")\nprint(f\"   🦴 Body Parts Groups: {list(BODY_PARTS.keys())}\")\nprint(f\"   🔗 Skeleton Connections: {len(SKELETON_CONNECTIONS)}\")\n\n# แสดงตารางสรุป Keypoints\nprint(\"\\n\" + \"=\" * 70)\nprint(\"📋 KEYPOINT REFERENCE TABLE\")\nprint(\"=\" * 70)\nkeypoint_df = pd.DataFrame({\n    'Index': range(len(KEYPOINT_NAMES)),\n    'Keypoint Name': KEYPOINT_NAMES,\n    'Thai Name': ['จมูก', 'ตาซ้าย', 'ตาขวา', 'หูซ้าย', 'หูขวา',\n                  'ไหล่ซ้าย', 'ไหล่ขวา', 'ข้อศอกซ้าย', 'ข้อศอกขวา',\n                  'ข้อมือซ้าย', 'ข้อมือขวา', 'สะโพกซ้าย', 'สะโพกขวา',\n                  'เข่าซ้าย', 'เข่าขวา', 'ข้อเท้าซ้าย', 'ข้อเท้าขวา']\n})\nprint(keypoint_df.to_string(index=False))\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 2: Exploratory Data Analysis (EDA)\n#\n# ในส่วนนี้เราจะสำรวจข้อมูลเพื่อเข้าใจลักษณะและการกระจายตัวของข้อมูล\n#\n# ### 2.1 Action Distribution Analysis\n#\n# **อธิบาย:** วิเคราะห์การกระจายตัวของ Actions ต่างๆ ในข้อมูล\n#\n# **ทำไมต้องวิเคราะห์ Action Distribution?**\n# - เพื่อดูความสมดุลของข้อมูล (Class Balance)\n# - ระบุ Actions ที่มีข้อมูลมาก/น้อย\n# - วางแผนการวิเคราะห์และ sampling strategy\n\n# %%\n# =====================================================\n# STEP 2.1: Action Distribution Analysis\n# =====================================================\n# 📝 Description: \n# วิเคราะห์จำนวนและสัดส่วนของแต่ละ Action\n# เพื่อเข้าใจความสมดุลของข้อมูล\n#\n# 📊 Output:\n# - ตารางแสดงจำนวน frames ต่อ action\n# - Bar chart แสดงการกระจายตัว\n# - Pie chart แสดงสัดส่วน\n#\n# 💡 Insight:\n# ถ้า action ใดมีข้อมูลน้อยเกินไป อาจต้องระวัง\n# เรื่อง bias ในการวิเคราะห์\n# =====================================================\n\n# นับจำนวนแต่ละ Action\naction_counts = df_pose['action'].value_counts()\naction_percentages = df_pose['action'].value_counts(normalize=True) * 100\n\nprint(\"=\" * 70)\nprint(\"🎯 ACTION DISTRIBUTION\")\nprint(\"=\" * 70)\n\n# แสดงตาราง\naction_summary = pd.DataFrame({\n    'Action': action_counts.index,\n    'Count': action_counts.values,\n    'Percentage (%)': action_percentages.values.round(2)\n})\nprint(\"\\n📋 Summary Table:\")\nprint(action_summary.to_string(index=False))\n\n# สร้าง Visualization\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\n# Bar Chart\ncolors = plt.cm.Set3(np.linspace(0, 1, len(action_counts)))\nbars = axes[0].barh(action_counts.index, action_counts.values, color=colors)\naxes[0].set_xlabel('Number of Frames')\naxes[0].set_title('📊 Action Distribution (Bar Chart)')\naxes[0].invert_yaxis()\n\n# เพิ่มตัวเลขบน bar\nfor bar, count in zip(bars, action_counts.values):\n    axes[0].text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2, \n                 f'{count:,}', va='center', fontsize=10)\n\n# Pie Chart\naxes[1].pie(action_counts.values, labels=action_counts.index, \n            autopct='%1.1f%%', colors=colors, startangle=90)\naxes[1].set_title('📊 Action Distribution (Pie Chart)')\n\nplt.tight_layout()\nplt.show()\n\n# %% [markdown]\n# ### 2.2 Person ID Analysis\n#\n# **อธิบาย:** วิเคราะห์จำนวน Frames ต่อ Person และ Person ต่อ Action\n#\n# **คำถามที่ต้องการตอบ:**\n# 1. แต่ละ Person มีกี่ frames?\n# 2. แต่ละ Action มี Person กี่คนที่ทำ?\n# 3. ข้อมูลกระจายตัวสม่ำเสมอหรือไม่?\n\n# %%\n# =====================================================\n# STEP 2.2: Person ID Analysis\n# =====================================================\n# 📝 Description: \n# วิเคราะห์ว่าแต่ละ Person มีกี่ frames\n# และแต่ละ Action มี Person กี่คน\n#\n# 🎯 Purpose:\n# - ตรวจสอบความครอบคลุมของข้อมูล\n# - หา Persons ที่มีข้อมูลเพียงพอสำหรับการวิเคราะห์\n# - ดูความหลากหลายของผู้ทำแต่ละ action\n# =====================================================\n\n# นับ frames ต่อ person\nperson_frame_counts = df_pose.groupby('person_id').size().reset_index(name='frame_count')\nperson_frame_counts = person_frame_counts.sort_values('frame_count', ascending=False)\n\nprint(\"=\" * 70)\nprint(\"👥 PERSON ID ANALYSIS\")\nprint(\"=\" * 70)\n\nprint(\"\\n📋 Frames per Person (Top 10):\")\nprint(person_frame_counts.head(10).to_string(index=False))\n\nprint(f\"\\n📊 Statistics:\")\nprint(f\"   Total Unique Persons: {len(person_frame_counts)}\")\nprint(f\"   Max Frames per Person: {person_frame_counts['frame_count'].max():,}\")\nprint(f\"   Min Frames per Person: {person_frame_counts['frame_count'].min():,}\")\nprint(f\"   Mean Frames per Person: {person_frame_counts['frame_count'].mean():.2f}\")\nprint(f\"   Median Frames per Person: {person_frame_counts['frame_count'].median():.2f}\")\n\n# Person distribution per action\nprint(\"\\n\" + \"=\" * 70)\nprint(\"🎯 PERSONS PER ACTION\")\nprint(\"=\" * 70)\n\nperson_per_action = df_pose.groupby('action')['person_id'].nunique().reset_index()\nperson_per_action.columns = ['Action', 'Unique Persons']\nprint(person_per_action.to_string(index=False))\n\n# Visualization\nfig, axes = plt.subplots(1, 2, figsize=(14, 5))\n\n# Frames per person (top 15)\ntop_persons = person_frame_counts.head(15)\naxes[0].bar(top_persons['person_id'].astype(str), top_persons['frame_count'], \n            color='steelblue', edgecolor='navy')\naxes[0].set_xlabel('Person ID')\naxes[0].set_ylabel('Number of Frames')\naxes[0].set_title('👥 Frames per Person (Top 15)')\naxes[0].tick_params(axis='x', rotation=45)\n\n# Persons per action\ncolors = plt.cm.Set2(np.linspace(0, 1, len(person_per_action)))\naxes[1].bar(person_per_action['Action'], person_per_action['Unique Persons'], \n            color=colors, edgecolor='black')\naxes[1].set_xlabel('Action')\naxes[1].set_ylabel('Number of Unique Persons')\naxes[1].set_title('🎯 Unique Persons per Action')\naxes[1].tick_params(axis='x', rotation=45)\n\nplt.tight_layout()\nplt.show()\n\n# %% [markdown]\n# ### 2.3 Select 3 Representative Person IDs for Analysis\n#\n# **อธิบาย:** เลือก 3 Person IDs ที่มีข้อมูลครบถ้วนสำหรับการวิเคราะห์เปรียบเทียบ\n#\n# **เกณฑ์การเลือก:**\n# 1. มี labeled frames มากที่สุด (ไม่ใช่ 'Unknown')\n# 2. ปรากฏในหลาย actions\n# 3. มีช่วงเวลาต่อเนื่องยาวพอสำหรับ time series analysis\n#\n# ```\n# ┌─────────────────────────────────────────────┐\n# │          SELECTION CRITERIA                 │\n# ├─────────────────────────────────────────────┤\n# │ ✓ Labeled frames (not 'Unknown')           │\n# │ ✓ Multiple actions performed               │\n# │ ✓ Continuous time series                   │\n# │ ✓ High confidence keypoints                │\n# └─────────────────────────────────────────────┘\n# ```\n\n# %%\n# =====================================================\n# STEP 2.3: Select 3 Representative Person IDs\n# =====================================================\n# 📝 Description: \n# เลือก 3 Person IDs ที่มี frames มากที่สุด\n# และปรากฏใน Action labels (ไม่ใช่ Unknown)\n# เพื่อใช้ในการวิเคราะห์เปรียบเทียบ\n#\n# 🔄 Process:\n# 1. กรองเฉพาะ frames ที่มี label\n# 2. นับ frames ต่อ person\n# 3. เลือก top 3 persons\n# 4. แสดงรายละเอียด\n#\n# 💡 Note: \n# การเลือก persons ที่มีข้อมูลมากช่วยให้การวิเคราะห์\n# มีความน่าเชื่อถือมากขึ้น\n# =====================================================\n\n# กรองเฉพาะ frames ที่มี label (ไม่ใช่ Unknown)\ndf_labeled = df_pose[df_pose['action'] != 'Unknown'].copy()\n\nprint(f\"📊 Filtered Data: {len(df_labeled):,} frames with labels (excluded 'Unknown')\")\n\n# นับ frames ต่อ person สำหรับข้อมูลที่มี label\nperson_labeled_counts = df_labeled.groupby('person_id').agg({\n    'frame_idx': 'count',\n    'action': lambda x: x.nunique()\n}).reset_index()\nperson_labeled_counts.columns = ['person_id', 'labeled_frames', 'actions_count']\nperson_labeled_counts = person_labeled_counts.sort_values('labeled_frames', ascending=False)\n\nprint(\"\\n\" + \"=\" * 70)\nprint(\"🎯 SELECTING REPRESENTATIVE PERSON IDs\")\nprint(\"=\" * 70)\n\nprint(\"\\n📋 Persons with Labeled Frames (Top 10):\")\nprint(person_labeled_counts.head(10).to_string(index=False))\n\n# เลือก 3 person IDs ที่มี frames มากที่สุด\nSELECTED_PERSON_IDS = person_labeled_counts.head(3)['person_id'].tolist()\n\nprint(f\"\\n✅ Selected Person IDs for Analysis: {SELECTED_PERSON_IDS}\")\n\n# แสดงรายละเอียดของ selected persons\nprint(\"\\n\" + \"=\" * 70)\nprint(\"📊 SELECTED PERSONS DETAILS\")\nprint(\"=\" * 70)\n\nfor pid in SELECTED_PERSON_IDS:\n    person_data = df_labeled[df_labeled['person_id'] == pid]\n    actions = person_data['action'].unique()\n    print(f\"\\n👤 Person ID: {pid}\")\n    print(f\"   📈 Total Labeled Frames: {len(person_data):,}\")\n    print(f\"   🎯 Actions: {list(actions)}\")\n    print(f\"   📅 Frame Range: {person_data['frame_idx'].min()} - {person_data['frame_idx'].max()}\")\n    print(f\"   ⏱️  Time Range: {person_data['timestamp'].min():.2f}s - {person_data['timestamp'].max():.2f}s\")\n    print(f\"   📊 Duration: {person_data['timestamp'].max() - person_data['timestamp'].min():.2f}s\")\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 3: Time Series Analysis\n#\n# ในส่วนนี้เราจะวิเคราะห์การเคลื่อนไหวตามเวลา (Time Series) ของ keypoints ต่างๆ\n#\n# ### 3.1 Keypoint Position Time Series\n#\n# **อธิบาย:** วิเคราะห์การเปลี่ยนแปลงตำแหน่ง Keypoints ตามเวลาสำหรับแต่ละ Action\n#\n# **Key Keypoints สำหรับการวิเคราะห์ท่ามวย:**\n#\n# | Keypoint | ความสำคัญ |\n# |----------|----------|\n# | right_wrist | การเคลื่อนไหวของหมัดขวา |\n# | left_wrist | การเคลื่อนไหวของหมัดซ้าย |\n# | right_ankle | การเคลื่อนไหวของเตะขวา |\n# | left_ankle | การเคลื่อนไหวของเตะซ้าย |\n# | nose | การเคลื่อนไหวของศีรษะ/ลำตัว |\n#\n# **ภาพประกอบ Coordinate System:**\n# ```\n# (0,0) ────────────────→ X (เพิ่มไปทางขวา)\n#   │\n#   │    ○ nose\n#   │   /|\\\n#   │  ○ │ ○\n#   │    │\n#   ↓    │\n#   Y    │\n# (เพิ่มลงล่าง)\n# ```\n\n# %%\n# =====================================================\n# STEP 3.1: Keypoint Position Time Series Analysis\n# =====================================================\n# 📝 Description: \n# วิเคราะห์การเคลื่อนที่ของ keypoints หลักๆ ตามเวลา\n# เพื่อเข้าใจ pattern การเคลื่อนไหวในแต่ละท่า\n# \n# 🎯 Key Keypoints:\n# - wrists: การเคลื่อนไหวของมือ (สำคัญสำหรับหมัด)\n# - ankles: การเคลื่อนไหวของเท้า (สำคัญสำหรับการเตะ)\n# - nose: การเคลื่อนไหวของศีรษะ\n#\n# 📊 Output:\n# - กราฟ X coordinate ตามเวลา\n# - กราฟ Y coordinate ตามเวลา\n# - แยกสีตาม action\n# =====================================================\n\n# Keypoints ที่น่าสนใจสำหรับการวิเคราะห์ท่ามวย\nANALYSIS_KEYPOINTS = ['right_wrist', 'left_wrist', 'right_ankle', 'left_ankle', 'nose']\n\ndef plot_keypoint_timeseries(df, person_id, keypoints, figsize=(16, 12)):\n    \"\"\"\n    สร้างกราฟ Time Series ของตำแหน่ง Keypoints\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose\n    person_id : int - Person ID ที่ต้องการวิเคราะห์\n    keypoints : list - รายชื่อ keypoints\n    figsize : tuple - ขนาดรูป\n    \n    Returns:\n    --------\n    fig : matplotlib figure object\n    \"\"\"\n    # กรองข้อมูลสำหรับ person นี้\n    person_data = df[df['person_id'] == person_id].copy()\n    person_data = person_data.sort_values('frame_idx')\n    \n    # สร้าง figure\n    fig, axes = plt.subplots(len(keypoints), 2, figsize=figsize)\n    fig.suptitle(f'👤 Keypoint Time Series - Person ID: {person_id}', \n                 fontsize=16, fontweight='bold')\n    \n    # สร้าง color map สำหรับ actions\n    actions = person_data['action'].unique()\n    action_colors = dict(zip(actions, plt.cm.Set2(np.linspace(0, 1, len(actions)))))\n    \n    for idx, kpt in enumerate(keypoints):\n        # Plot X coordinate\n        ax_x = axes[idx, 0]\n        for action in actions:\n            action_data = person_data[person_data['action'] == action]\n            ax_x.scatter(action_data['timestamp'], action_data[f'{kpt}_x'], \n                        label=action, alpha=0.6, s=2, color=action_colors[action])\n        \n        ax_x.set_ylabel(f'{kpt}\\nX Position (pixels)')\n        ax_x.set_title(f'{kpt.replace(\"_\", \" \").title()} - X Coordinate')\n        if idx == 0:\n            ax_x.legend(loc='upper right', fontsize=8, markerscale=5)\n        \n        # Plot Y coordinate\n        ax_y = axes[idx, 1]\n        for action in actions:\n            action_data = person_data[person_data['action'] == action]\n            ax_y.scatter(action_data['timestamp'], action_data[f'{kpt}_y'], \n                        label=action, alpha=0.6, s=2, color=action_colors[action])\n        \n        ax_y.set_ylabel(f'{kpt}\\nY Position (pixels)')\n        ax_y.set_title(f'{kpt.replace(\"_\", \" \").title()} - Y Coordinate')\n        \n        # กลับแกน Y เพราะ pixel coordinate มี Y เพิ่มขึ้นเมื่อไปทางล่าง\n        ax_y.invert_yaxis()\n    \n    # Set x labels for bottom row\n    axes[-1, 0].set_xlabel('Time (seconds)')\n    axes[-1, 1].set_xlabel('Time (seconds)')\n    \n    plt.tight_layout()\n    return fig\n\n# สร้างกราฟสำหรับ Person ID แรกที่เลือก\nprint(\"=\" * 70)\nprint(\"📈 KEYPOINT TIME SERIES VISUALIZATION\")\nprint(\"=\" * 70)\nprint(\"\\n💡 Interpretation Guide:\")\nprint(\"   - แต่ละสีแทน action ที่แตกต่างกัน\")\nprint(\"   - แกน X: เวลา (วินาที)\")\nprint(\"   - แกน Y ซ้าย: ตำแหน่ง X (pixels) - ซ้าย/ขวา\")\nprint(\"   - แกน Y ขวา: ตำแหน่ง Y (pixels) - บน/ล่าง (inverted)\")\nprint(\"\\n\" + \"-\" * 70)\n\nfor pid in SELECTED_PERSON_IDS[:1]:  # แสดงเฉพาะ person แรก\n    print(f\"\\n📊 Generating plot for Person ID: {pid}\")\n    fig = plot_keypoint_timeseries(df_labeled, pid, ANALYSIS_KEYPOINTS)\n    plt.show()\n\n# %% [markdown]\n# ### 3.2 Compare Keypoint Trajectories Across Actions\n#\n# **อธิบาย:** เปรียบเทียบ Trajectory ของ Keypoints ในแต่ละ Action\n#\n# **วัตถุประสงค์:**\n# - เห็นความแตกต่างของการเคลื่อนไหวในแต่ละท่า\n# - เปรียบเทียบระหว่าง persons\n# - หา pattern ที่เป็นเอกลักษณ์ของแต่ละ action\n#\n# **การอ่านกราฟ:**\n# ```\n# ─── เส้นทึบ: X coordinate\n# --- เส้นประ: Y coordinate\n# สีต่างกัน: Person ID ต่างกัน\n# ```\n\n# %%\n# =====================================================\n# STEP 3.2: Compare Keypoint Trajectories Across Actions\n# =====================================================\n# 📝 Description: \n# เปรียบเทียบการเคลื่อนที่ของ keypoints\n# ในแต่ละ action เพื่อหา pattern ที่แตกต่างกัน\n#\n# 🔍 What to look for:\n# - รูปแบบการเคลื่อนไหวที่ซ้ำๆ (repetitive patterns)\n# - ความแตกต่างระหว่าง actions\n# - ความคล้ายคลึงระหว่าง persons ทำท่าเดียวกัน\n# =====================================================\n\ndef plot_action_comparison(df, person_ids, keypoint, figsize=(16, 10)):\n    \"\"\"\n    เปรียบเทียบ trajectory ของ keypoint ในแต่ละ action\n    สำหรับ selected persons\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose\n    person_ids : list - รายการ person IDs\n    keypoint : str - ชื่อ keypoint\n    figsize : tuple - ขนาดรูป\n    \n    Returns:\n    --------\n    fig : matplotlib figure object\n    \"\"\"\n    # กรองข้อมูล\n    df_selected = df[df['person_id'].isin(person_ids)].copy()\n    actions = [a for a in df_selected['action'].unique() if a != 'Unknown']\n    \n    # สร้าง figure\n    n_actions = len(actions)\n    n_cols = 3\n    n_rows = (n_actions + n_cols - 1) // n_cols\n    \n    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)\n    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]\n    \n    fig.suptitle(f'🎯 {keypoint.replace(\"_\", \" \").title()} Trajectory Comparison\\n'\n                 f'Selected Persons: {person_ids}', \n                 fontsize=14, fontweight='bold')\n    \n    # Color map สำหรับ persons\n    person_colors = dict(zip(person_ids, ['#e74c3c', '#3498db', '#2ecc71']))\n    \n    for idx, action in enumerate(actions):\n        ax = axes[idx]\n        action_data = df_selected[df_selected['action'] == action]\n        \n        for pid in person_ids:\n            person_data = action_data[action_data['person_id'] == pid]\n            if len(person_data) > 0:\n                # Normalize time to start from 0\n                person_data = person_data.sort_values('timestamp')\n                time_normalized = person_data['timestamp'] - person_data['timestamp'].min()\n                \n                ax.plot(time_normalized, person_data[f'{keypoint}_x'], \n                       label=f'Person {pid} (X)', linestyle='-', alpha=0.8,\n                       color=person_colors[pid])\n                ax.plot(time_normalized, person_data[f'{keypoint}_y'], \n                       label=f'Person {pid} (Y)', linestyle='--', alpha=0.8,\n                       color=person_colors[pid])\n        \n        ax.set_title(action.replace('_', ' '), fontsize=11)\n        ax.set_xlabel('Normalized Time (s)')\n        ax.set_ylabel('Position (pixels)')\n        \n        if idx == 0:\n            ax.legend(fontsize=8, loc='upper right')\n    \n    # ซ่อน axes ที่ไม่ใช้\n    for idx in range(len(actions), len(axes)):\n        axes[idx].set_visible(False)\n    \n    plt.tight_layout()\n    return fig\n\n# สร้างกราฟเปรียบเทียบสำหรับ right_wrist (สำคัญสำหรับหมัด)\nprint(\"=\" * 70)\nprint(\"📊 ACTION TRAJECTORY COMPARISON\")\nprint(\"=\" * 70)\nprint(\"\\n💡 Reading Guide:\")\nprint(\"   ─── เส้นทึบ: X coordinate (ซ้าย-ขวา)\")\nprint(\"   --- เส้นประ: Y coordinate (บน-ล่าง)\")\nprint(\"   🔴 แดง: Person 1\")\nprint(\"   🔵 น้ำเงิน: Person 2\") \nprint(\"   🟢 เขียว: Person 3\")\nprint(\"\\n\" + \"-\" * 70)\n\nfor kpt in ['right_wrist', 'right_ankle']:\n    print(f\"\\n📊 Analyzing: {kpt.replace('_', ' ').title()}\")\n    fig = plot_action_comparison(df_labeled, SELECTED_PERSON_IDS, kpt)\n    plt.show()\n\n# %% [markdown]\n# ### 3.3 Velocity Analysis\n#\n# **อธิบาย:** วิเคราะห์ความเร็วของ Keypoints เพื่อหา pattern การเคลื่อนไหวที่รวดเร็ว\n#\n# **สูตรคำนวณความเร็ว:**\n# ```\n# velocity = √((dx/dt)² + (dy/dt)²)\n#\n# โดยที่:\n# - dx = การเปลี่ยนแปลงตำแหน่ง X\n# - dy = การเปลี่ยนแปลงตำแหน่ง Y  \n# - dt = การเปลี่ยนแปลงเวลา\n# ```\n#\n# **การตีความ:**\n# - **ความเร็วสูง** = การเคลื่อนไหวที่รวดเร็ว (หมัด, เตะ)\n# - **ความเร็วต่ำ** = การเคลื่อนไหวช้าๆ (ท่าตั้งรับ, การเตรียมตัว)\n#\n# ```\n#        High Velocity              Low Velocity\n#            ↗                          →\n#           /                           \n#          /                            \n#         ○ ─→ ───→ ────→              ○ → ○ → ○\n#        หมัด/เตะ                      ท่าตั้งรับ\n# ```\n\n# %%\n# =====================================================\n# STEP 3.3: Velocity Analysis\n# =====================================================\n# 📝 Description: \n# คำนวณความเร็วของ keypoints\n# ความเร็ว = การเปลี่ยนแปลงตำแหน่งต่อเวลา\n# \n# 📐 Formula: \n# velocity = sqrt((dx/dt)^2 + (dy/dt)^2)\n# \n# 🎯 Interpretation:\n# - ความเร็วสูง = การเคลื่อนไหวที่รวดเร็ว (หมัด, เตะ)\n# - ความเร็วต่ำ = การเคลื่อนไหวช้าๆ (ท่าตั้งรับ)\n#\n# 🔧 Smoothing:\n# ใช้ uniform_filter1d เพื่อลด noise ในข้อมูล velocity\n# =====================================================\n\ndef calculate_velocity(df, person_id, keypoint, window_size=3):\n    \"\"\"\n    คำนวณความเร็วของ keypoint\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose\n    person_id : int - Person ID\n    keypoint : str - ชื่อ keypoint\n    window_size : int - ขนาด window สำหรับ smoothing\n    \n    Returns:\n    --------\n    DataFrame พร้อมคอลัมน์ velocity และ velocity_smooth\n    \"\"\"\n    # กรองและ sort ข้อมูล\n    person_data = df[df['person_id'] == person_id].copy()\n    person_data = person_data.sort_values('timestamp').reset_index(drop=True)\n    \n    # คำนวณ dx, dy, dt\n    person_data['dx'] = person_data[f'{keypoint}_x'].diff()\n    person_data['dy'] = person_data[f'{keypoint}_y'].diff()\n    person_data['dt'] = person_data['timestamp'].diff()\n    \n    # คำนวณ velocity (Euclidean distance per time)\n    person_data['velocity'] = np.sqrt(\n        (person_data['dx'] / person_data['dt'])**2 + \n        (person_data['dy'] / person_data['dt'])**2\n    )\n    \n    # Smooth velocity ด้วย moving average\n    if window_size > 1 and len(person_data) > window_size:\n        person_data['velocity_smooth'] = uniform_filter1d(\n            person_data['velocity'].fillna(0), size=window_size\n        )\n    else:\n        person_data['velocity_smooth'] = person_data['velocity']\n    \n    return person_data\n\ndef plot_velocity_analysis(df, person_ids, keypoint, figsize=(16, 10)):\n    \"\"\"\n    สร้างกราฟวิเคราะห์ความเร็วเปรียบเทียบระหว่าง actions\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose\n    person_ids : list - รายการ person IDs\n    keypoint : str - ชื่อ keypoint\n    figsize : tuple - ขนาดรูป\n    \n    Returns:\n    --------\n    fig : matplotlib figure object\n    \"\"\"\n    actions = [a for a in df['action'].unique() if a != 'Unknown']\n    \n    fig, axes = plt.subplots(len(person_ids), 1, figsize=figsize, sharex=True)\n    if len(person_ids) == 1:\n        axes = [axes]\n    \n    fig.suptitle(f'⚡ Velocity Analysis - {keypoint.replace(\"_\", \" \").title()}\\n'\n                 f'Higher velocity = Faster movement (e.g., punch, kick)', \n                 fontsize=14, fontweight='bold')\n    \n    # Color map สำหรับ actions\n    action_colors = dict(zip(actions, plt.cm.tab10(np.linspace(0, 1, len(actions)))))\n    \n    for idx, pid in enumerate(person_ids):\n        ax = axes[idx]\n        \n        # คำนวณ velocity\n        vel_data = calculate_velocity(df, pid, keypoint, window_size=5)\n        vel_data = vel_data[vel_data['action'] != 'Unknown']\n        \n        # Plot velocity for each action\n        for action in actions:\n            action_data = vel_data[vel_data['action'] == action]\n            if len(action_data) > 0:\n                ax.plot(action_data['timestamp'], action_data['velocity_smooth'],\n                       label=action, alpha=0.7, linewidth=0.8,\n                       color=action_colors[action])\n        \n        ax.set_ylabel(f'Person {pid}\\nVelocity (px/s)')\n        ax.set_ylim(0, vel_data['velocity_smooth'].quantile(0.99) * 1.1)\n        \n        if idx == 0:\n            ax.legend(loc='upper right', fontsize=8, ncol=2)\n    \n    axes[-1].set_xlabel('Time (seconds)')\n    \n    plt.tight_layout()\n    return fig\n\n# วิเคราะห์ความเร็วสำหรับ wrists\nprint(\"=\" * 70)\nprint(\"⚡ VELOCITY ANALYSIS\")\nprint(\"=\" * 70)\nprint(\"\\n💡 Interpretation:\")\nprint(\"   📈 Peak สูง = การเคลื่อนไหวเร็ว (หมัด/เตะ)\")\nprint(\"   📉 ค่าต่ำ = การเคลื่อนไหวช้า (เตรียมตัว/ตั้งรับ)\")\nprint(\"\\n\" + \"-\" * 70)\n\nfor kpt in ['right_wrist', 'left_wrist']:\n    print(f\"\\n📊 Velocity analysis for: {kpt.replace('_', ' ').title()}\")\n    fig = plot_velocity_analysis(df_labeled, SELECTED_PERSON_IDS, kpt)\n    plt.show()\n\n# %% [markdown]\n# ### 3.4 Velocity Statistics per Action\n#\n# **อธิบาย:** สรุปสถิติความเร็วเฉลี่ยของแต่ละ Action เพื่อจัดอันดับท่าที่เคลื่อนไหวเร็วที่สุด\n#\n# **สถิติที่คำนวณ:**\n# - **Mean Velocity**: ความเร็วเฉลี่ย - บอกความเร็วโดยทั่วไปของท่า\n# - **Max Velocity**: ความเร็วสูงสุด - บอกความเร็วในจังหวะที่เร็วที่สุด\n# - **Std Velocity**: ค่าเบี่ยงเบนมาตรฐาน - บอกความแปรปรวนของการเคลื่อนไหว\n\n# %%\n# =====================================================\n# STEP 3.4: Velocity Statistics per Action\n# =====================================================\n# 📝 Description: \n# คำนวณและเปรียบเทียบสถิติความเร็ว\n# ของแต่ละ action เพื่อระบุท่าที่เคลื่อนไหวเร็ว/ช้า\n#\n# 📊 Statistics computed:\n# - Mean Velocity: ความเร็วเฉลี่ย\n# - Max Velocity: ความเร็วสูงสุด\n# - Std Velocity: ค่าเบี่ยงเบนมาตรฐาน\n# - Median Velocity: ค่ามัธยฐาน\n# =====================================================\n\ndef calculate_action_velocity_stats(df, person_ids, keypoints):\n    \"\"\"\n    คำนวณสถิติความเร็วต่อ action\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose\n    person_ids : list - รายการ person IDs\n    keypoints : list - รายการ keypoints\n    \n    Returns:\n    --------\n    DataFrame ของสถิติความเร็ว\n    \"\"\"\n    results = []\n    \n    for pid in person_ids:\n        for kpt in keypoints:\n            vel_data = calculate_velocity(df, pid, kpt, window_size=5)\n            vel_data = vel_data[vel_data['action'] != 'Unknown']\n            \n            # คำนวณสถิติต่อ action\n            for action in vel_data['action'].unique():\n                action_data = vel_data[vel_data['action'] == action]['velocity_smooth']\n                action_data = action_data.dropna()\n                \n                if len(action_data) > 0:\n                    results.append({\n                        'person_id': pid,\n                        'keypoint': kpt,\n                        'action': action,\n                        'mean_velocity': action_data.mean(),\n                        'max_velocity': action_data.max(),\n                        'std_velocity': action_data.std(),\n                        'median_velocity': action_data.median()\n                    })\n    \n    return pd.DataFrame(results)\n\n# คำนวณสถิติ\nvelocity_stats = calculate_action_velocity_stats(\n    df_labeled, \n    SELECTED_PERSON_IDS, \n    ['right_wrist', 'left_wrist', 'right_ankle', 'left_ankle']\n)\n\nprint(\"=\" * 70)\nprint(\"📊 VELOCITY STATISTICS BY ACTION\")\nprint(\"=\" * 70)\n\n# เฉลี่ยข้าม persons และ keypoints\naction_velocity_summary = velocity_stats.groupby('action').agg({\n    'mean_velocity': 'mean',\n    'max_velocity': 'mean',\n    'std_velocity': 'mean'\n}).round(2)\n\naction_velocity_summary = action_velocity_summary.sort_values('mean_velocity', ascending=False)\n\nprint(\"\\n📋 Average Velocity Statistics by Action:\")\nprint(\"   (Averaged across all persons and keypoints)\")\nprint(\"\\n\" + action_velocity_summary.to_string())\n\nprint(\"\\n💡 Interpretation:\")\nprint(\"   🥇 Actions ที่อยู่ด้านบน = เคลื่อนไหวเร็วกว่า (หมัด/เตะ)\")\nprint(\"   🥉 Actions ที่อยู่ด้านล่าง = เคลื่อนไหวช้ากว่า (ตั้งรับ)\")\n\n# Visualization\nfig, axes = plt.subplots(1, 2, figsize=(14, 6))\n\n# Mean velocity by action\ncolors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(action_velocity_summary)))[::-1]\nbars = axes[0].barh(action_velocity_summary.index, \n                    action_velocity_summary['mean_velocity'],\n                    color=colors, edgecolor='black')\naxes[0].set_xlabel('Mean Velocity (pixels/second)')\naxes[0].set_title('📊 Mean Velocity by Action\\n(Higher = Faster movement)')\naxes[0].invert_yaxis()\n\n# เพิ่มค่าบน bar\nfor bar, val in zip(bars, action_velocity_summary['mean_velocity']):\n    axes[0].text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2,\n                 f'{val:.1f}', va='center', fontsize=9)\n\n# Max velocity by action\nbars = axes[1].barh(action_velocity_summary.index, \n                    action_velocity_summary['max_velocity'],\n                    color=colors, edgecolor='black')\naxes[1].set_xlabel('Max Velocity (pixels/second)')\naxes[1].set_title('📊 Max Velocity by Action\\n(Peak movement speed)')\naxes[1].invert_yaxis()\n\n# เพิ่มค่าบน bar\nfor bar, val in zip(bars, action_velocity_summary['max_velocity']):\n    axes[1].text(bar.get_width() + 5, bar.get_y() + bar.get_height()/2,\n                 f'{val:.1f}', va='center', fontsize=9)\n\nplt.tight_layout()\nplt.show()\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 4: Joint Angle Analysis\n#\n# ในส่วนนี้เราจะคำนวณและวิเคราะห์มุมข้อต่อ (Joint Angles) ซึ่งเป็นตัวแปรสำคัญในการวิเคราะห์ท่าทาง\n#\n# ### 4.1 Understanding Joint Angles\n#\n# **Joint Angle คืออะไร?**\n# - มุมที่เกิดจากข้อต่อ 3 จุด (เช่น ไหล่-ข้อศอก-ข้อมือ)\n# - ใช้ในการวิเคราะห์ท่าทางและการเคลื่อนไหว\n# - สำคัญมากในกีฬาและการวิเคราะห์การเคลื่อนไหว\n#\n# **Angles ที่จะคำนวณ:**\n#\n# | Angle | Points | ความหมาย |\n# |-------|--------|----------|\n# | Elbow Angle | shoulder-elbow-wrist | มุมงอข้อศอก |\n# | Knee Angle | hip-knee-ankle | มุมงอเข่า |\n# | Shoulder Angle | elbow-shoulder-hip | มุมยกแขน |\n# | Hip Angle | shoulder-hip-knee | มุมยกขา |\n# | Torso Lean | จากแนวตั้ง | มุมเอียงลำตัว |\n#\n# **ภาพประกอบการคำนวณมุม:**\n# ```\n#         p1 (shoulder)\n#          \\\n#           \\  ← angle\n#            \\\n#             p2 (elbow) ────── p3 (wrist)\n#\n# angle = arctan2(|cross(v1,v2)|, dot(v1,v2))\n# โดย v1 = p1 - p2, v2 = p3 - p2\n# ```\n#\n# **ตัวอย่างค่ามุม:**\n# ```\n# Elbow Angle:\n# - 180° = แขนเหยียดตรง\n# - 90°  = งอ 90 องศา\n# - 45°  = งอมาก\n#\n#   180°           90°            45°\n#   ────○         ○              ○\n#       │          \\            /\n#       │           \\          /\n#       ○            ○        ○\n# ```\n\n# %%\n# =====================================================\n# STEP 4.1: Joint Angle Calculation Functions\n# =====================================================\n# 📝 Description: \n# สร้างฟังก์ชันคำนวณมุมข้อต่อ\n# \n# 📐 Mathematical Principle:\n# ใช้ dot product และ cross product\n# angle = atan2(|cross_product|, dot_product)\n# \n# 📊 Output:\n# มุมเป็นองศา (degrees) ตั้งแต่ 0° ถึง 180°\n#\n# 💡 Note:\n# - 180° = เหยียดตรง\n# - 90° = งอฉาก\n# - 0° = พับชิด\n# =====================================================\n\ndef calculate_angle(p1, p2, p3):\n    \"\"\"\n    คำนวณมุมที่จุด p2 จากเส้น p1-p2 และ p2-p3\n    \n    Parameters:\n    -----------\n    p1, p2, p3 : np.array - พิกัด (x, y) ของแต่ละจุด\n    \n    Returns:\n    --------\n    angle : float - มุมเป็นองศา (0-180)\n    \n    Diagram:\n           p1\n            \\\n             \\  angle\n              p2-------p3\n    \n    Formula:\n    angle = arctan2(|v1 × v2|, v1 · v2)\n    \"\"\"\n    # Vector จาก p2 ไป p1 และ p3\n    v1 = p1 - p2\n    v2 = p3 - p2\n    \n    # ตรวจสอบว่ามีค่า valid หรือไม่ (0 = keypoint not detected)\n    if np.any(p1 == 0) or np.any(p2 == 0) or np.any(p3 == 0):\n        return np.nan\n    \n    # คำนวณ dot product: v1 · v2 = |v1||v2|cos(θ)\n    dot = np.dot(v1, v2)\n    \n    # คำนวณ cross product magnitude: |v1 × v2| = |v1||v2|sin(θ)\n    cross = np.cross(v1, v2)\n    \n    # คำนวณมุม (radians) ด้วย atan2 เพื่อได้มุมที่ถูกต้องในทุก quadrant\n    angle_rad = np.arctan2(np.abs(cross), dot)\n    \n    # แปลงเป็น degrees\n    angle_deg = np.degrees(angle_rad)\n    \n    return angle_deg\n\ndef calculate_all_angles(row):\n    \"\"\"\n    คำนวณมุมข้อต่อทั้งหมดสำหรับ 1 row ของข้อมูล\n    \n    Parameters:\n    -----------\n    row : pandas Series - 1 row ของ DataFrame\n    \n    Returns:\n    --------\n    dict : dictionary ของมุมต่างๆ\n    \"\"\"\n    angles = {}\n    \n    # Helper function ดึงพิกัด keypoint\n    def get_point(name):\n        return np.array([row[f'{name}_x'], row[f'{name}_y']])\n    \n    # ===== Elbow Angles (มุมข้อศอก) =====\n    # การงอข้อศอก: shoulder-elbow-wrist\n    angles['left_elbow_angle'] = calculate_angle(\n        get_point('left_shoulder'),\n        get_point('left_elbow'),\n        get_point('left_wrist')\n    )\n    \n    angles['right_elbow_angle'] = calculate_angle(\n        get_point('right_shoulder'),\n        get_point('right_elbow'),\n        get_point('right_wrist')\n    )\n    \n    # ===== Knee Angles (มุมเข่า) =====\n    # การงอเข่า: hip-knee-ankle\n    angles['left_knee_angle'] = calculate_angle(\n        get_point('left_hip'),\n        get_point('left_knee'),\n        get_point('left_ankle')\n    )\n    \n    angles['right_knee_angle'] = calculate_angle(\n        get_point('right_hip'),\n        get_point('right_knee'),\n        get_point('right_ankle')\n    )\n    \n    # ===== Shoulder Angles (มุมไหล่) =====\n    # การยกแขน: elbow-shoulder-hip\n    angles['left_shoulder_angle'] = calculate_angle(\n        get_point('left_elbow'),\n        get_point('left_shoulder'),\n        get_point('left_hip')\n    )\n    \n    angles['right_shoulder_angle'] = calculate_angle(\n        get_point('right_elbow'),\n        get_point('right_shoulder'),\n        get_point('right_hip')\n    )\n    \n    # ===== Hip Angles (มุมสะโพก) =====\n    # การยกขา: shoulder-hip-knee\n    angles['left_hip_angle'] = calculate_angle(\n        get_point('left_shoulder'),\n        get_point('left_hip'),\n        get_point('left_knee')\n    )\n    \n    angles['right_hip_angle'] = calculate_angle(\n        get_point('right_shoulder'),\n        get_point('right_hip'),\n        get_point('right_knee')\n    )\n    \n    # ===== Torso Lean Angle (มุมเอียงลำตัว) =====\n    # คำนวณจากเส้นที่เชื่อม shoulder center กับ hip center\n    shoulder_mid = (get_point('left_shoulder') + get_point('right_shoulder')) / 2\n    hip_mid = (get_point('left_hip') + get_point('right_hip')) / 2\n    \n    # มุมของลำตัวเทียบกับแนวตั้ง (vertical)\n    vertical = np.array([0, -1])  # ขึ้นบน (ใน pixel coord, Y ลดลง = ขึ้นบน)\n    torso_vector = shoulder_mid - hip_mid\n    \n    if np.linalg.norm(torso_vector) > 0:\n        torso_vector_norm = torso_vector / np.linalg.norm(torso_vector)\n        dot = np.dot(torso_vector_norm, vertical)\n        angles['torso_lean_angle'] = np.degrees(np.arccos(np.clip(dot, -1, 1)))\n    else:\n        angles['torso_lean_angle'] = np.nan\n    \n    return angles\n\nprint(\"✅ Angle calculation functions defined!\")\nprint(\"\\n📐 Angles to be calculated:\")\nprint(\"   ┌────────────────────────────────────────────┐\")\nprint(\"   │ Angle Name          │ Points               │\")\nprint(\"   ├────────────────────────────────────────────┤\")\nprint(\"   │ Elbow Angle         │ shoulder-elbow-wrist │\")\nprint(\"   │ Knee Angle          │ hip-knee-ankle       │\")\nprint(\"   │ Shoulder Angle      │ elbow-shoulder-hip   │\")\nprint(\"   │ Hip Angle           │ shoulder-hip-knee    │\")\nprint(\"   │ Torso Lean          │ from vertical        │\")\nprint(\"   └────────────────────────────────────────────┘\")\n\n# %% [markdown]\n# ### 4.2 Calculate Angles for All Data\n#\n# **อธิบาย:** คำนวณมุมข้อต่อทั้งหมดสำหรับทุก frame ในข้อมูล\n#\n# **ขั้นตอน:**\n# 1. ใช้ฟังก์ชัน `calculate_all_angles()` กับทุก row\n# 2. เพิ่มคอลัมน์มุมเข้าไปใน DataFrame\n# 3. ตรวจสอบผลลัพธ์\n\n# %%\n# =====================================================\n# STEP 4.2: Calculate Angles for All Data\n# =====================================================\n# 📝 Description: \n# คำนวณมุมข้อต่อทั้งหมดสำหรับทุก frame\n#\n# ⏱️ Note: \n# การคำนวณนี้อาจใช้เวลาสักครู่ (30 วินาที - 2 นาที)\n# ขึ้นอยู่กับจำนวนข้อมูล\n#\n# 🔄 Process:\n# 1. Apply calculate_all_angles() to each row\n# 2. Convert results to DataFrame\n# 3. Merge with original data\n# =====================================================\n\nprint(\"=\" * 70)\nprint(\"📐 CALCULATING JOINT ANGLES\")\nprint(\"=\" * 70)\nprint(f\"\\n⏳ Processing {len(df_labeled):,} frames... This may take a moment.\")\n\n# คำนวณมุมทั้งหมด\nimport time\nstart_time = time.time()\n\n# Apply function to each row และแปลงเป็น DataFrame\nangles_list = df_labeled.apply(calculate_all_angles, axis=1).tolist()\nangles_df = pd.DataFrame(angles_list)\n\n# รวมกับข้อมูลเดิม\ndf_with_angles = pd.concat([df_labeled.reset_index(drop=True), angles_df], axis=1)\n\nelapsed_time = time.time() - start_time\nprint(f\"✅ Calculation completed in {elapsed_time:.2f} seconds\")\n\n# แสดงรายชื่อ angle columns\nangle_columns = [col for col in df_with_angles.columns if '_angle' in col]\nprint(f\"\\n📊 Angle columns added: {len(angle_columns)}\")\nfor col in angle_columns:\n    print(f\"   - {col}\")\n\n# แสดงตัวอย่างข้อมูล\nprint(\"\\n\" + \"=\" * 70)\nprint(\"📋 SAMPLE DATA WITH ANGLES\")\nprint(\"=\" * 70)\ndisplay(df_with_angles[['frame_idx', 'person_id', 'action'] + angle_columns].head(10))\n\n# แสดงสถิติเบื้องต้นของมุม\nprint(\"\\n\" + \"=\" * 70)\nprint(\"📊 ANGLE STATISTICS SUMMARY\")\nprint(\"=\" * 70)\nprint(df_with_angles[angle_columns].describe().round(2))\n\n# %% [markdown]\n# ### 4.3 Angle Time Series Visualization\n#\n# **อธิบาย:** แสดงการเปลี่ยนแปลงของมุมข้อต่อตามเวลา\n#\n# **การอ่านกราฟ:**\n# - แกน Y: มุมเป็นองศา (0° - 180°)\n# - เส้นประสีเทา 90°: มุมฉาก (reference line)\n# - สีต่างกัน: actions ต่างกัน\n#\n# **Pattern ที่ควรสังเกต:**\n# - **หมัดตรง**: elbow angle เพิ่มขึ้นอย่างรวดเร็ว (แขนเหยียด)\n# - **เตะ**: knee angle เพิ่มขึ้น + hip angle เปลี่ยน\n# - **ท่าตั้งรับ**: มุมค่อนข้างคงที่\n\n# %%\n# =====================================================\n# STEP 4.3: Angle Time Series Visualization\n# =====================================================\n# 📝 Description: \n# สร้างกราฟแสดงการเปลี่ยนแปลงของมุมข้อต่อ\n# ตามเวลาสำหรับแต่ละ action\n#\n# 📊 Graph elements:\n# - Y-axis: Angle in degrees (0° - 180°)\n# - Gray dashed line at 90°: right angle reference\n# - Different colors: different actions\n#\n# 💡 What to look for:\n# - Rapid changes = fast movements\n# - Stable values = static poses\n# - Periodic patterns = repetitive movements\n# =====================================================\n\ndef plot_angle_timeseries(df, person_id, angles, figsize=(16, 12)):\n    \"\"\"\n    สร้างกราฟ Time Series ของมุมข้อต่อ\n    \n    Parameters:\n    -----------\n    df : DataFrame - ข้อมูล pose พร้อมมุม\n    person_id : int - Person ID\n    angles : list - รายชื่อ angle columns\n    figsize : tuple - ขนาดรูป\n    \n    Returns:\n    --------\n    fig : matplotlib figure object\n    \"\"\"\n    person_data = df[df['person_id'] == person_id].copy()\n    person_data = person_data.sort_values('timestamp')\n    \n    # สร้าง figure\n    fig, axes = plt.subplots(len(angles), 1, figsize=figsize, sharex=True)\n    if len(angles) == 1:\n        axes = [axes]\n    \n    fig.suptitle(f'📐 Joint Angle Time Series - Person ID: {person_id}', \n                 fontsize=14, fontweight='bold')\n    \n    # Color map สำหรับ actions\n    actions = [a for a in person_data['action'].unique() if a != 'Unknown']\n    action_colors = dict(zip(actions, plt.cm.Set2(np.linspace(0, 1, len(actions)))))\n    \n    for idx, angle_name in enumerate(angles):\n        ax = axes[idx]\n        \n        for action in actions:\n            action_data = person_data[person_data['action'] == action]\n            if len(action_data) > 0:\n                ax.scatter(action_data['timestamp'], action_data[angle_name],\n                          label=action, alpha=0.5, s=3, color=action_colors[action])\n        \n        # Format axis\n        ax.set_ylabel(f'{angle_name.replace(\"_\", \" \").title()}\\n(degrees)')\n        ax.set_ylim(0, 180)\n        \n        # Reference lines\n        ax.axhline(y=90, color='gray', linestyle='--', alpha=0.5, label='90°' if idx == 0 else '')\n        ax.axhline(y=180, color='gray', linestyle=':', alpha=0.3)\n        ax.axhline(y=0, color='gray', linestyle=':', alpha=0.3)\n        \n        if idx == 0:\n            ax.legend(loc='upper right', fontsize=8, ncol=3, markerscale=3)\n    \n    axes[-1].set_xlabel('Time (seconds)')\n    \n    plt.tight_layout()\n    return fig\n\n# Angles ที่จะวิเคราะห์\nANALYSIS_ANGLES = ['right_elbow_angle', 'left_elbow_angle', \n                   'right_knee_angle', 'left_knee_angle',\n                   'torso_lean_angle']\n\nprint(\"=\" * 70)\nprint(\"📐 ANGLE TIME SERIES VISUALIZATION\")\nprint(\"=\" * 70)\nprint(\"\\n💡 Reading Guide:\")\nprint(\"   - Y-axis: Angle (0° to 180°)\")\nprint(\"   - 180° = fully extended (straight)\")\nprint(\"   - 90° = right angle\")\nprint(\"   - 0° = fully flexed\")\nprint(\"   - Gray dashed line = 90° reference\")\nprint(\"\\n\" + \"-\" * 70)\n\nfor pid in SELECTED_PERSON_IDS[:1]:  # แสดงเฉพาะ person แรก\n    print(f\"\\n📊 Generating plot for Person ID: {pid}\")\n    fig = plot_angle_timeseries(df_with_angles, pid, ANALYSIS_ANGLES)\n    plt.show()\n\n# %% [markdown]\n# ### 4.4 Angle Statistics per Action\n#\n# **อธิบาย:** เปรียบเทียบค่าเฉลี่ยและการกระจายของมุมในแต่ละ Action\n#\n# **Box Plot Interpretation:**\n# ```\n#     ┬── Maximum (Q3 + 1.5*IQR)\n#     │\n#   ┌─┼─┐ ── Q3 (75th percentile)\n#   │ │ │\n#   │ ├─┤ ── Median (Q2, 50th percentile)\n#   │ │ │\n#   └─┼─┘ ── Q1 (25th percentile)\n#     │\n#     ┴── Minimum (Q1 - 1.5*IQR)\n#     \n#     ○ ── Outliers\n# ```\n\n# %%\n# =====================================================\n# STEP 4.4: Angle Statistics per Action\n# =====================================================\n# 📝 Description: \n# คำนวณและเปรียบเทียบสถิติมุมของแต่ละ action\n# เพื่อระบุลักษณะเฉพาะของแต่ละท่า\n#\n# 📊 Statistics:\n# - Mean: ค่าเฉลี่ย\n# - Std: ค่าเบี่ยงเบนมาตรฐาน\n# - Min/Max: ค่าต่ำสุด/สูงสุด\n#\n# 📈 Visualization:\n# Box plot แสดงการกระจายของมุมในแต่ละ action\n# =====================================================\n\n# คำนวณสถิติมุมต่อ action\nangle_stats = df_with_angles.groupby('action')[angle_columns].agg(['mean', 'std', 'min', 'max'])\nangle_stats = angle_stats.round(2)\n\nprint(\"=\" * 70)\nprint(\"📊 ANGLE STATISTICS BY ACTION\")\nprint(\"=\" * 70)\n\n# แสดงค่าเฉลี่ยมุม\nprint(\"\\n📋 Mean Angle by Action (degrees):\")\nmean_angles = df_with_angles.groupby('action')[angle_columns].mean().round(2)\nprint(mean_angles.to_string())\n\n# Visualization - Box plots for key angles\nfig, axes = plt.subplots(2, 3, figsize=(16, 10))\naxes = axes.flatten()\n\nkey_angles = ['right_elbow_angle', 'left_elbow_angle', \n              'right_knee_angle', 'left_knee_angle',\n              'right_shoulder_angle', 'torso_lean_angle']\n\nactions_list = [a for a in df_with_angles['action'].unique() if a != 'Unknown']\n\nfor idx, angle in enumerate(key_angles):\n    ax = axes[idx]\n    \n    # สร้าง box plot\n    data_for_plot = [df_with_angles[df_with_angles['action'] == action][angle].dropna() \n                     for action in actions_list]\n    \n    bp = ax.boxplot(data_for_plot, labels=[a.replace('_', '\\n')[:15] for a in actions_list],\n                    patch_artist=True)\n    \n    # สีสำหรับ box\n    colors = plt.cm.Set3(np.linspace(0, 1, len(actions_list)))\n    for patch, color in zip(bp['boxes'], colors):\n        patch.set_facecolor(color)\n        patch.set_alpha(0.7)\n    \n    ax.set_ylabel('Angle (degrees)')\n    ax.set_title(f'📐 {angle.replace(\"_\", \" \").title()}')\n    ax.tick_params(axis='x', rotation=45)\n    ax.axhline(y=90, color='red', linestyle='--', alpha=0.5, linewidth=1)\n    ax.set_ylim(0, 180)\n\nplt.suptitle('📊 Joint Angle Distribution by Action\\n(Red dashed line = 90°)', \n             fontsize=14, fontweight='bold')\nplt.tight_layout()\nplt.show()\n\n# %% [markdown]\n# ### 4.5 Angle Heatmap Comparison\n#\n# **อธิบาย:** สร้าง Heatmap เปรียบเทียบค่าเฉลี่ยมุมของแต่ละ Action\n#\n# **การอ่าน Heatmap:**\n# - **สีแดงเข้ม**: มุมมาก (ใกล้ 180° = เหยียด)\n# - **สีน้ำเงินเข้ม**: มุมน้อย (ใกล้ 0° = งอ)\n# - **ตัวเลขในช่อง**: ค่าเฉลี่ยมุม (องศา)\n\n# %%\n# =====================================================\n# STEP 4.5: Angle Heatmap Comparison\n# =====================================================\n# 📝 Description: \n# สร้าง Heatmap แสดงค่าเฉลี่ยมุมของแต่ละ action\n# ทำให้เห็นภาพรวมได้ง่าย\n#\n# 🎨 Color interpretation:\n# - Dark red: large angle (near 180° = extended)\n# - Dark blue: small angle (near 0° = flexed)\n# - Numbers: mean angle in degrees\n# =====================================================\n\n# คำนวณค่าเฉลี่ยมุมต่อ action\nmean_angles_matrix = df_with_angles[df_with_angles['action'] != 'Unknown'].groupby('action')[angle_columns].mean()\n\n# สร้าง Heatmap\nfig, ax = plt.subplots(figsize=(14, 8))\n\n# แปลงชื่อ columns ให้อ่านง่าย\ndisplay_columns = [col.replace('_', ' ').replace(' angle', '').title() \n                   for col in angle_columns]\n\nsns.heatmap(mean_angles_matrix.values, \n            annot=True, \n            fmt='.1f',\n            cmap='RdYlBu_r',\n            xticklabels=display_columns,\n            yticklabels=mean_angles_matrix.index,\n            vmin=0, \n            vmax=180,\n            cbar_kws={'label': 'Angle (degrees)'},\n            ax=ax,\n            linewidths=0.5,\n            linecolor='white')\n\nax.set_title('📊 Mean Joint Angles Heatmap by Action\\n'\n             '(Darker red = larger angle/extended, Darker blue = smaller angle/flexed)', \n             fontsize=14, fontweight='bold')\nax.set_xlabel('Joint Angle')\nax.set_ylabel('Action')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n💡 Heatmap Interpretation Guide:\")\nprint(\"   🔴 Red colors (high values): Extended/straightened joints\")\nprint(\"   🔵 Blue colors (low values): Flexed/bent joints\")\nprint(\"   📊 Numbers: Mean angle in degrees for each action-joint combination\")\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 5: Action Comparison Analysis\n#\n# ### 5.1 Create Action Feature Summary\n#\n# **อธิบาย:** สร้างตาราง Feature สรุปลักษณะเด่นของแต่ละ Action\n\n# %%\n# =====================================================\n# STEP 5.1: Create Action Feature Summary\n# =====================================================\n# 📝 Description: \n# สรุปค่า features หลักๆ ของแต่ละ action\n# เพื่อใช้ในการเปรียบเทียบและ classification\n# =====================================================\n\n# สร้าง feature summary ต่อ action\nactions_to_compare = [a for a in df_with_angles['action'].unique() if a != 'Unknown']\n\naction_features = []\n\nfor action in actions_to_compare:\n    action_data = df_with_angles[df_with_angles['action'] == action]\n    \n    features = {\n        'action': action,\n        'frame_count': len(action_data),\n        'unique_persons': action_data['person_id'].nunique(),\n        \n        # Angle means\n        'mean_right_elbow': action_data['right_elbow_angle'].mean(),\n        'mean_left_elbow': action_data['left_elbow_angle'].mean(),\n        'mean_right_knee': action_data['right_knee_angle'].mean(),\n        'mean_left_knee': action_data['left_knee_angle'].mean(),\n        'mean_torso_lean': action_data['torso_lean_angle'].mean(),\n        \n        # Angle std (variability)\n        'std_right_elbow': action_data['right_elbow_angle'].std(),\n        'std_left_elbow': action_data['left_elbow_angle'].std(),\n        'std_right_knee': action_data['right_knee_angle'].std(),\n        'std_left_knee': action_data['left_knee_angle'].std(),\n    }\n    \n    action_features.append(features)\n\naction_features = pd.DataFrame(action_features)\n\nprint(\"=\" * 70)\nprint(\"📊 ACTION FEATURE SUMMARY\")\nprint(\"=\" * 70)\nprint(\"\\n📋 Features per Action:\")\ndisplay(action_features.round(2))\n\n# %% [markdown]\n# ### 5.2 Radar Chart Comparison\n#\n# **อธิบาย:** สร้าง Radar Chart (Spider Chart) เปรียบเทียบลักษณะของแต่ละ Action\n#\n# **การอ่าน Radar Chart:**\n# - แต่ละแกน = 1 feature (normalized)\n# - พื้นที่กว้าง = ค่า feature สูงในหลายมิติ\n# - รูปร่างต่างกัน = ลักษณะท่าแตกต่างกัน\n\n# %%\n# =====================================================\n# STEP 5.2: Radar Chart Comparison\n# =====================================================\n# 📝 Description: \n# สร้าง Radar Chart เปรียบเทียบ features ของแต่ละ action\n#\n# 📊 Chart interpretation:\n# - Each axis = 1 normalized feature\n# - Larger area = higher values in multiple dimensions\n# - Different shapes = different action characteristics\n# =====================================================\n\ndef create_radar_chart(df_features, actions_to_plot, features_to_plot, figsize=(10, 10)):\n    \"\"\"\n    สร้าง Radar Chart สำหรับเปรียบเทียบ actions\n    \n    Parameters:\n    -----------\n    df_features : DataFrame - ข้อมูล features ต่อ action\n    actions_to_plot : list - actions ที่ต้องการ plot\n    features_to_plot : list - features ที่ต้องการ plot\n    figsize : tuple - ขนาดรูป\n    \n    Returns:\n    --------\n    fig : matplotlib figure object\n    \"\"\"\n    # กรองและ normalize ข้อมูล\n    df_plot = df_features[df_features['action'].isin(actions_to_plot)].copy()\n    \n    # Normalize features to 0-1 scale\n    for feature in features_to_plot:\n        min_val = df_plot[feature].min()\n        max_val = df_plot[feature].max()\n        if max_val > min_val:\n            df_plot[f'{feature}_norm'] = (df_plot[feature] - min_val) / (max_val - min_val)\n        else:\n            df_plot[f'{feature}_norm'] = 0.5\n    \n    # สร้าง radar chart\n    num_vars = len(features_to_plot)\n    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()\n    angles += angles[:1]  # close the polygon\n    \n    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(polar=True))\n    \n    colors = plt.cm.Set2(np.linspace(0, 1, len(actions_to_plot)))\n    \n    for idx, action in enumerate(actions_to_plot):\n        action_row = df_plot[df_plot['action'] == action]\n        if len(action_row) == 0:\n            continue\n            \n        values = [action_row[f'{f}_norm'].values[0] for f in features_to_plot]\n        values += values[:1]  # close the polygon\n        \n        ax.plot(angles, values, 'o-', linewidth=2, label=action, color=colors[idx])\n        ax.fill(angles, values, alpha=0.25, color=colors[idx])\n    \n    # Set labels\n    feature_labels = [f.replace('mean_', '').replace('_', ' ').title() for f in features_to_plot]\n    ax.set_xticks(angles[:-1])\n    ax.set_xticklabels(feature_labels, size=10)\n    \n    ax.set_ylim(0, 1)\n    ax.set_title('🎯 Action Feature Comparison (Radar Chart)\\n'\n                 'Values normalized to 0-1 scale', fontsize=14, fontweight='bold', y=1.08)\n    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))\n    \n    return fig\n\n# เลือก features สำหรับ radar chart\nradar_features = ['mean_right_elbow', 'mean_left_elbow', \n                  'mean_right_knee', 'mean_left_knee', 'mean_torso_lean']\n\nprint(\"=\" * 70)\nprint(\"🎯 RADAR CHART COMPARISON\")\nprint(\"=\" * 70)\n\n# สร้าง radar chart (เลือก 5 actions แรก)\nactions_for_radar = actions_to_compare[:5]\nprint(f\"\\n📊 Comparing actions: {actions_for_radar}\")\n\nfig = create_radar_chart(action_features, actions_for_radar, radar_features)\nplt.show()\n\nprint(\"\\n💡 Interpretation:\")\nprint(\"   - แต่ละแกนแทน 1 feature (normalized 0-1)\")\nprint(\"   - รูปร่างต่างกัน = ท่าทางต่างกัน\")\nprint(\"   - พื้นที่มาก = มุมโดยรวมมาก (เหยียดมากกว่า)\")\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 6: Advanced Analysis\n#\n# ### 6.1 Correlation Analysis Between Angles\n#\n# **อธิบาย:** วิเคราะห์ความสัมพันธ์ระหว่างมุมข้อต่อต่างๆ\n\n# %%\n# =====================================================\n# STEP 6.1: Correlation Analysis Between Angles\n# =====================================================\n# 📝 Description: \n# วิเคราะห์ความสัมพันธ์ระหว่างมุมข้อต่อต่างๆ\n#\n# 📊 Correlation interpretation:\n# - +1: perfect positive correlation\n# - 0: no correlation\n# - -1: perfect negative correlation\n# =====================================================\n\n# คำนวณ correlation matrix\nangle_corr = df_with_angles[angle_columns].corr()\n\n# สร้าง heatmap\nfig, ax = plt.subplots(figsize=(10, 8))\n\nmask = np.triu(np.ones_like(angle_corr, dtype=bool))  # แสดงเฉพาะครึ่งล่าง\n\nsns.heatmap(angle_corr, \n            mask=mask,\n            annot=True, \n            fmt='.2f',\n            cmap='coolwarm',\n            center=0,\n            vmin=-1, vmax=1,\n            square=True,\n            ax=ax,\n            linewidths=0.5)\n\n# แปลงชื่อให้อ่านง่าย\nlabels = [col.replace('_angle', '').replace('_', ' ').title() for col in angle_columns]\nax.set_xticklabels(labels, rotation=45, ha='right')\nax.set_yticklabels(labels, rotation=0)\n\nax.set_title('📊 Correlation Matrix: Joint Angles\\n'\n             '(Red = positive correlation, Blue = negative correlation)', \n             fontsize=14, fontweight='bold')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n💡 Correlation Insights:\")\nprint(\"   🔴 Red (positive): Angles tend to increase together\")\nprint(\"   🔵 Blue (negative): One increases while other decreases\")\nprint(\"   ⚪ White (zero): No relationship\")\n\n# %% [markdown]\n# ### 6.2 Symmetry Analysis (Left vs Right)\n#\n# **อธิบาย:** เปรียบเทียบความสมมาตรของร่างกายซ้าย-ขวา\n\n# %%\n# =====================================================\n# STEP 6.2: Symmetry Analysis (Left vs Right)\n# =====================================================\n# 📝 Description: \n# วิเคราะห์ความสมมาตรของร่างกายซ้าย-ขวา\n# ถ้า difference มาก = ท่าทางไม่สมมาตร (เช่น หมัดข้างเดียว)\n# =====================================================\n\n# คำนวณ difference ระหว่างซ้าย-ขวา\ndf_with_angles['elbow_diff'] = np.abs(df_with_angles['right_elbow_angle'] - df_with_angles['left_elbow_angle'])\ndf_with_angles['knee_diff'] = np.abs(df_with_angles['right_knee_angle'] - df_with_angles['left_knee_angle'])\ndf_with_angles['shoulder_diff'] = np.abs(df_with_angles['right_shoulder_angle'] - df_with_angles['left_shoulder_angle'])\ndf_with_angles['hip_diff'] = np.abs(df_with_angles['right_hip_angle'] - df_with_angles['left_hip_angle'])\n\n# สรุปตาม action\nsymmetry_cols = ['elbow_diff', 'knee_diff', 'shoulder_diff', 'hip_diff']\nsymmetry_by_action = df_with_angles.groupby('action')[symmetry_cols].mean().round(2)\nsymmetry_by_action = symmetry_by_action.sort_values('elbow_diff', ascending=False)\n\nprint(\"=\" * 70)\nprint(\"⚖️ SYMMETRY ANALYSIS (Left-Right Difference)\")\nprint(\"=\" * 70)\nprint(\"\\n📋 Mean Angle Difference by Action:\")\nprint(\"   (Higher value = more asymmetric)\")\nprint(\"\\n\" + symmetry_by_action.to_string())\n\n# Visualization\nfig, ax = plt.subplots(figsize=(12, 6))\n\nx = np.arange(len(symmetry_by_action))\nwidth = 0.2\n\nbars1 = ax.bar(x - 1.5*width, symmetry_by_action['elbow_diff'], width, label='Elbow', color='#e74c3c')\nbars2 = ax.bar(x - 0.5*width, symmetry_by_action['knee_diff'], width, label='Knee', color='#3498db')\nbars3 = ax.bar(x + 0.5*width, symmetry_by_action['shoulder_diff'], width, label='Shoulder', color='#2ecc71')\nbars4 = ax.bar(x + 1.5*width, symmetry_by_action['hip_diff'], width, label='Hip', color='#9b59b6')\n\nax.set_xlabel('Action')\nax.set_ylabel('Mean Angle Difference (degrees)')\nax.set_title('⚖️ Body Symmetry Analysis by Action\\n(Higher = More Asymmetric)', fontsize=14, fontweight='bold')\nax.set_xticks(x)\nax.set_xticklabels([a.replace('_', '\\n')[:15] for a in symmetry_by_action.index], rotation=45, ha='right')\nax.legend()\nax.axhline(y=10, color='gray', linestyle='--', alpha=0.5, label='10° threshold')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\n💡 Interpretation:\")\nprint(\"   📊 ค่าสูง = ท่าไม่สมมาตร (เช่น หมัดข้างเดียว, เตะข้างเดียว)\")\nprint(\"   📊 ค่าต่ำ = ท่าสมมาตร (เช่น ท่าตั้งรับ, ท่าเตรียม)\")\n\n# %% [markdown]\n# ---\n#\n# ## 📚 Part 7: Summary and Conclusions\n#\n# ### 7.1 Generate Final Report\n#\n# **อธิบาย:** สรุปผลการวิเคราะห์ทั้งหมดเป็น report\n\n# %%\n# =====================================================\n# STEP 7.1: Generate Final Summary Report\n# =====================================================\n# 📝 Description: \n# สรุปผลการวิเคราะห์ทั้งหมดเป็น report\n# =====================================================\n\nprint(\"=\" * 70)\nprint(\"📋 FINAL ANALYSIS REPORT\")\nprint(\"=\" * 70)\n\nprint(f\"\"\"\n🎯 DATASET OVERVIEW\n─────────────────────────────────────────\n• Total Frames: {len(df_pose):,}\n• Total Labeled Frames: {len(df_labeled):,}\n• Unique Persons: {df_pose['person_id'].nunique()}\n• Actions Analyzed: {len(actions_to_compare)}\n• Selected Persons for Analysis: {SELECTED_PERSON_IDS}\n\n📊 ACTIONS SUMMARY\n─────────────────────────────────────────\"\"\")\n\nfor action in actions_to_compare[:6]:  # แสดง 6 actions แรก\n    action_data = df_with_angles[df_with_angles['action'] == action]\n    print(f\"\\n🥊 {action.replace('_', ' ')}\")\n    print(f\"   Frames: {len(action_data):,}\")\n    print(f\"   Mean Right Elbow Angle: {action_data['right_elbow_angle'].mean():.1f}°\")\n    print(f\"   Mean Right Knee Angle: {action_data['right_knee_angle'].mean():.1f}°\")\n    print(f\"   Mean Torso Lean: {action_data['torso_lean_angle'].mean():.1f}°\")\n\nprint(f\"\"\"\n\n📈 KEY FINDINGS\n─────────────────────────────────────────\n1. ✅ Actions ที่ต้องการการเหยียดแขน (elbow angle สูง) เด่นชัด\n2. ✅ ท่าที่ต้องงอเข่า (knee angle ต่ำ) สามารถแยกแยะได้ดี\n3. ✅ มุมลำตัว (torso lean) ช่วยบ่งบอกท่าทางโจมตี/ป้องกัน\n4. ✅ Velocity analysis ช่วยระบุช่วงเวลาของการเคลื่อนไหวเร็ว\n5. ✅ Symmetry analysis ช่วยระบุท่าที่ใช้แขน/ขาข้างเดียว\n\n─────────────────────────────────────────\n📌 RECOMMENDATIONS FOR FURTHER ANALYSIS\n─────────────────────────────────────────\n1. 🤖 ใช้ Machine Learning เพื่อ classify actions อัตโนมัติ\n2. 📈 วิเคราะห์ temporal patterns ด้วย DTW หรือ LSTM\n3. 📊 สร้าง feature vectors จาก angles สำหรับ clustering\n4. 🎥 เพิ่ม 3D reconstruction ถ้ามีข้อมูลจากหลายกล้อง\n5. ⏱️  วิเคราะห์ timing และ rhythm ของการเคลื่อนไหว\n\"\"\")\n\n# %% [markdown]\n# ### 7.2 Export Processed Data\n#\n# **อธิบาย:** Export ข้อมูลที่ประมวลผลแล้วสำหรับใช้งานต่อ\n#\n# **ไฟล์ที่จะ Export:**\n# 1. `pose_data_with_angles.csv` - ข้อมูลพร้อมมุมที่คำนวณ\n# 2. `action_features_summary.csv` - สรุป features ต่อ action\n# 3. `velocity_statistics.csv` - สถิติความเร็ว\n\n# %%\n# =====================================================\n# STEP 7.2: Export Processed Data\n# =====================================================\n# 📝 Description: \n# บันทึกข้อมูลที่ประมวลผลแล้ว\n# รวมถึง angles ที่คำนวณ\n#\n# 📁 Output files:\n# - pose_data_with_angles.csv\n# - action_features_summary.csv\n# - velocity_statistics.csv\n#\n# ⚠️ Note: Uncomment the code below to export\n# =====================================================\n\n# # Export DataFrame พร้อม angles\n# output_filename = 'pose_data_with_angles.csv'\n# df_with_angles.to_csv(output_filename, index=False)\n# print(f\"✅ Exported: {output_filename}\")\n\n# # Export action features\n# features_filename = 'action_features_summary.csv'\n# action_features.to_csv(features_filename, index=False)\n# print(f\"✅ Exported: {features_filename}\")\n\n# # Export velocity statistics\n# velocity_filename = 'velocity_statistics.csv'\n# velocity_stats.to_csv(velocity_filename, index=False)\n# print(f\"✅ Exported: {velocity_filename}\")\n\nprint(\"=\" * 70)\nprint(\"💾 EXPORT OPTIONS (Currently commented out)\")\nprint(\"=\" * 70)\nprint(\"\"\"\n📁 Files available for export:\n\n1. pose_data_with_angles.csv\n   - Original pose data + calculated joint angles\n   - Size: ~{:,} rows × {:,} columns\n   \n2. action_features_summary.csv\n   - Aggregated features per action\n   - Size: {:,} rows × {:,} columns\n   \n3. velocity_statistics.csv\n   - Velocity statistics per action/person/keypoint\n   - Size: {:,} rows × {:,} columns\n\n💡 To export, uncomment the code above and run again.\n\"\"\".format(len(df_with_angles), len(df_with_angles.columns),\n           len(action_features), len(action_features.columns),\n           len(velocity_stats), len(velocity_stats.columns)))\n\nprint(\"\\n\" + \"=\" * 70)\nprint(\"🎉 LAB COMPLETED SUCCESSFULLY!\")\nprint(\"=\" * 70)\nprint(\"\"\"\nคุณได้เรียนรู้:\n✅ การโหลดและสำรวจข้อมูล Pose Estimation\n✅ การวิเคราะห์ Time Series ของ keypoint positions\n✅ การคำนวณ Velocity และวิเคราะห์ความเร็ว\n✅ การคำนวณ Joint Angles จาก keypoint coordinates\n✅ การเปรียบเทียบ Actions ด้วย visualization ต่างๆ\n✅ การวิเคราะห์ Symmetry และ Correlation\n\n🚀 Next Steps:\n- ลองนำ features ไปใช้ train ML model\n- เพิ่ม features อื่นๆ เช่น acceleration, jerk\n- วิเคราะห์ temporal patterns ด้วย sequence models\n\"\"\")\n\n# %% [markdown]\n# ---\n#\n# ## 📖 Additional Resources\n#\n# **สำหรับศึกษาเพิ่มเติม:**\n#\n# ### 1. Pose Estimation\n# - [COCO Keypoint Detection](https://cocodataset.org/#keypoints-2020) - มาตรฐาน keypoint format\n# - [YOLOv11 Documentation](https://docs.ultralytics.com/) - เอกสาร pose estimation\n# - [MediaPipe Pose](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker) - alternative pose estimation\n#\n# ### 2. Time Series Analysis\n# - [Pandas Time Series](https://pandas.pydata.org/docs/user_guide/timeseries.html) - การจัดการข้อมูลเวลา\n# - [SciPy Signal Processing](https://docs.scipy.org/doc/scipy/reference/signal.html) - การประมวลผลสัญญาณ\n# - [Dynamic Time Warping](https://rtavenar.github.io/blog/dtw.html) - การเปรียบเทียบ time series\n#\n# ### 3. Joint Angle Analysis\n# - [Biomechanics Tutorials](https://biomechanics.stanford.edu/) - หลักการ biomechanics\n# - [OpenSim Documentation](https://opensim.stanford.edu/) - เครื่องมือวิเคราะห์การเคลื่อนไหว\n#\n# ### 4. Visualization\n# - [Matplotlib Gallery](https://matplotlib.org/stable/gallery/) - ตัวอย่างกราฟ\n# - [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html) - การสร้างกราฟสถิติ\n#\n# ### 5. Machine Learning for Action Recognition\n# - [scikit-learn](https://scikit-learn.org/) - ML algorithms\n# - [TensorFlow/Keras](https://www.tensorflow.org/) - Deep learning\n# - [PyTorch](https://pytorch.org/) - Deep learning alternative\n#\n# ---\n#\n# **End of Lab** 🎓"
    }
  ],
  "stop_reason": "end_turn",
 
}

In [ ]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

# %% [markdown]
# # 🥊 Pose Time Series Analysis Lab: การวิเคราะห์ท่ามวยไทยด้วย Time Series และ Angle Analysis
#
# ---
#
# ## 📋 Lab Overview
#
# **วัตถุประสงค์การเรียนรู้ (Learning Objectives):**
# 1. เข้าใจโครงสร้างข้อมูล Pose Estimation ในรูปแบบ Time Series
# 2. วิเคราะห์และเปรียบเทียบการเคลื่อนไหวของบุคคลต่างๆ ข้ามเวลา
# 3. คำนวณ Joint Angles จาก Keypoint Coordinates
# 4. สร้าง Visualization เพื่อเปรียบเทียบท่าทางต่างๆ (Actions)
# 5. วิเคราะห์ Pattern ของท่ามวยไทยผ่าน Time Series และ Angle Analysis
#
# **Prerequisites:**
# - Python 3.8+
# - ความเข้าใจพื้นฐานเรื่อง Pandas DataFrame
# - ความเข้าใจพื้นฐานเรื่อง Trigonometry (สำหรับการคำนวณมุม)
#
# **Estimated Time:** 2-3 hours
#
# ---

# %% [markdown]
# ## 📚 Part 1: Environment Setup และ Data Loading
#
# ในส่วนนี้เราจะเตรียมสภาพแวดล้อมสำหรับการวิเคราะห์ โดยการ import libraries ที่จำเป็น
# และโหลดข้อมูล Pose Estimation จากไฟล์ CSV
#
# ### 1.1 Import Libraries
#
# **อธิบาย:** นำเข้า libraries ที่จำเป็นสำหรับการวิเคราะห์ Time Series และการ

: 